In [74]:
import os
import re
import string
import random
from re import search
from nltk import bigrams
import nltk.classify.util
from nltk import FreqDist
from nltk.collocations import *
from nltk.corpus import stopwords
from nltk.corpus import movie_reviews
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.metrics import ConfusionMatrix
from nltk.tokenize import RegexpTokenizer
from nltk.classify import NaiveBayesClassifier

In [20]:
# prepare the list of stopwords
stopwords = stopwords.words("english")
stopwords.extend(['subject','to','cc','bcc','not','insert','shopping'])
newstopwords = [word for word in stopwords if word not in ['not', 'no', 'can','don', 't']]

In [21]:
rootdir = "corpus"
ham_list = []
spam_list = []
#specify how many files you want to read from the ham and spam
limit = 100

In [22]:
# define how the data will be read fro ham and spam
def read_mails(readFileLimit):
    # Loop through all the directories, sub directories and files in the above folder, and print them.

    # For files, print number of files.

    for directories, subdirs, files in os.walk(rootdir):
        print(directories, subdirs, len(files))

    # Same as before, but only print the ham and spam folders
    for directories, subdirs, files in os.walk(rootdir):
        if (os.path.split(directories)[1]  == 'ham'):
            print(directories, subdirs, len(files))

        if (os.path.split(directories)[1]  == 'spam'):
            print(directories, subdirs, len(files))
    
    # Same as before, but this time, read the files, and append them to the ham and spam list
    for directories, subdirs, files in os.walk(rootdir):
        if (os.path.split(directories)[1]  == 'ham'):
            for filename in files[:readFileLimit]:      
                with open(os.path.join(directories, filename), encoding="latin-1") as f:
                    data = f.read()
                    ham_list.append(data)

        if (os.path.split(directories)[1]  == 'spam'):
            for filename in files[:readFileLimit]:
                with open(os.path.join(directories, filename), encoding="latin-1") as f:
                    data = f.read()
                    spam_list.append(data)


In [23]:
# Reading the mails
read_mails(limit)
#print(ham_list[0:120])
#print(spam_list[0])

corpus ['ham', 'spam'] 2
corpus\ham [] 3672
corpus\spam [] 1500
corpus\ham [] 3672
corpus\spam [] 1500


In [24]:
# convert list of file data to combined single email string for ham
hamListToStr = ''.join([str(elem) for elem in ham_list]) 
hamListToStr = hamListToStr.lower()

In [25]:
# convert list of file data to combined single email string for spam
spamListToStr = ''.join([str(elem) for elem in spam_list]) 
spamListToStr = spamListToStr.lower()

In [26]:
# prepare to separate out all the emails
d = "subject"
ham_list_splittedByPattern = []
ham_finalMailList = []
spam_list_splittedByPattern = []
spam_finalMailList = []
patterns = ['am - - - - - - - - - - - - - - - - - - - - - - - - - - -', 'from :', 'to :', 'cc :', 'bcc :']

In [27]:
# separemoving ating each emails and removing the lines which contains above patterns
def prepare_separate_mails(msgType):
    globals()[msgType+"_list_splittedBySubject"] =  [d+e for e in globals()[msgType+"ListToStr"].split(d) if e]
    for mail in globals()[msgType+"_list_splittedBySubject"]:
        for pattern in patterns:
            #print('Looking for "%s" in "%s" ->' % (pattern, mail), end=' ')
            if re.search(pattern, mail):
                #print('print mail body "%s ->' % mail.split(pattern))
                globals()[msgType+"_list_splittedByPattern"] = mail.split(pattern)
                for mailString in globals()[msgType+"_list_splittedByPattern"]:
                    if d in mailString:
                        globals()[msgType+"_finalMailList"].append(mailString)
                break
            else:
                globals()[msgType+"_finalMailList"].append(mail)
                break

In [28]:
# prepared separated out email list for spam and ham
prepare_separate_mails("ham")
prepare_separate_mails("spam")

In [29]:
#print(ham_finalMailList)

In [30]:
#########################################################################################
####  Pre-processing the documents  ####
#########################################################################################
def pre_processing_documents(document):
  # "Pre_processing_documents"  
  # "create list of lower case words"
  word_list = re.split('\s+', document.lower())
  # punctuation and numbers to be removed
  punctuation = re.compile(r'[-+\*.?!/\%@#,":;()|0-9]')
  word_list = [punctuation.sub("", word) for word in word_list] 
  final_word_list = []
  for word in word_list:
    if word not in newstopwords:
      final_word_list.append(word)
  line = " ".join(final_word_list)
  return line 

In [31]:
#########################################################################################
####  get words/tokens from documents  ####
#########################################################################################
def get_words_from_phasedocs(docs):
  all_words = []
  for (words, sentiment) in docs:
    # more than 3 length
    possible_words = [x for x in words if len(x) >= 3]
    all_words.extend(possible_words)
  return all_words

def get_words_from_phasedocs_normal(docs):
  all_words = []
  for (words, sentiment) in docs:
    all_words.extend(words)
  return all_words  

# get all words from tokens
def get_words_from_test(lines):
  all_words = []
  for id,words in lines:
    all_words.extend(words)
  return all_words

In [32]:
#########################################################################################
####  Function writeFeatureSets to csv file ####
#########################################################################################
# Function writeFeatureSets:
# takes featuresets defined in the nltk and convert them to weka input csv file
#    any feature value in the featuresets should not contain ",", "'" or " itself
#    and write the file to the outpath location
#    outpath should include the name of the csv file
def writeFeatureSets(featuresets, outpath):
    # open outpath for writing
    f = open(outpath, 'w')
    # get the feature names from the feature dictionary in the first featureset
    featurenames = featuresets[0][0].keys()
    # create the first line of the file as comma separated feature names
    #    with the word class as the last feature name
    featurenameline = ''
    for featurename in featurenames:
        # replace forbidden characters with text abbreviations
        featurename = featurename.replace(',','CM')
        featurename = featurename.replace("'","DQ")
        featurename = featurename.replace('"','QU')
        featurenameline += featurename + ','
    featurenameline += 'class'
    # write this as the first line in the csv file
    f.write(featurenameline)
    f.write('\n')
    # convert each feature set to a line in the file with comma separated feature values,
    # each feature value is converted to a string 
    #   for booleans this is the words true and false
    #   for numbers, this is the string with the number
    for featureset in featuresets:
        featureline = ''
        for key in featurenames:
            featureline += str(featureset[0][key]) + ','
        if featureset[1] == 0:
          featureline += str("neg")
        elif featureset[1] == 1:
          featureline += str("sneg")
        elif featureset[1] == 2:
          featureline += str("neu")
        elif featureset[1] == 3:
          featureline += str("spos")
        elif featureset[1] == 4:
          featureline += str("pos")
        # write each feature set values to the file
        f.write(featureline)
        f.write('\n')
    f.close()

In [33]:
# define a feature definition function here
####################################################################
#### Get word-features, bag of words  ####
####################################################################
def get_word_features(wordlist):
  wordlist = nltk.FreqDist(wordlist)
  word_features = [w for (w, c) in wordlist.most_common(200)] 
  return word_features    

# define features (keywords) of a document for a BOW/unigram baseline
# each feature is 'contains(keyword)' and is true or false depending
# on whether that keyword is in the document
# define a feature definition function here
def normal_features(document, word_features):
  document_words = set(document)
  features = {}
  for word in word_features:
    features['contains({})'.format(word)] = (word in document_words)
  return features

In [65]:
# define a feature definition function here
####################################################################
#### Get bigram document features  ####
####################################################################
# define features that include words as before 
# add the most frequent significant bigrams
# this function takes the list of words in a document as an argument and returns a feature dictionary
# it depends on the variables word_features and bigram_features
def bigram_document_features(document, word_features,bigram_features):
  document_words = set(document)
  document_bigrams = nltk.bigrams(document)
  features = {}
  for word in word_features:
    features['contains({})'.format(word)] = (word in document_words)
  for bigram in bigram_features:
    features['bigram({} {})'.format(bigram[0], bigram[1])] = (bigram in document_bigrams)    
  return features
def get_biagram_features(tokens):
  bigram_measures = nltk.collocations.BigramAssocMeasures()
  td_finder = BigramCollocationFinder.from_words(tokens,window_size=3)
  td_scored = td_finder.score_ngrams(bigram_measures.raw_freq)
  # Print the first 50 bigram scores
  print("--------------frequency score--------------")
  for bscore in td_scored[:50]:
      print(bscore)
  # Apply a mininmum word frequency of five to The Dead and print the top 50 bigrams. 
  td_finder.apply_freq_filter(5)
  td_scored = td_finder.score_ngrams(bigram_measures.pmi)
  print("\n--------------pmi score--------------")
  for bscore in td_scored[:50]:
      print(bscore)
  #finder.apply_freq_filter(6)
  bigram_features = td_finder.nbest(bigram_measures.chi_sq, 3000)
  return bigram_features[:500]

In [35]:
####################################################################
#### Sentiment Lexicon: Subjectivity Count features  ####
####################################################################
# define features that include word counts of subjectivity words
# negative feature will have number of weakly negative words +
#    2 * number of strongly negative words
# positive feature has similar definition
#    not counting neutral words
# create your own path to the subjclues file
def readSubjectivity(path):
  flexicon = open(path, 'r')
  # initialize an empty dictionary
  sldict = { }
  for line in flexicon:
    fields = line.split()   # default is to split on whitespace
    # split each field on the '=' and keep the second part as the value
    strength = fields[0].split("=")[1]
    word = fields[2].split("=")[1]
    posTag = fields[3].split("=")[1]
    stemmed = fields[4].split("=")[1]
    polarity = fields[5].split("=")[1]
    if (stemmed == 'y'):
      isStemmed = True
    else:
      isStemmed = False
    # put a dictionary entry with the word as the keyword
    #     and a list of the other values
    sldict[word] = [strength, posTag, isStemmed, polarity]
  return sldict

In [36]:
SLpath = "SentimentLexicons/subjclueslen1-HLTEMNLP05.tff"
SL = readSubjectivity(SLpath)

In [37]:
def SL_features(document, word_features, SL):
  document_words = set(document)
  features = {}
  for word in word_features:
    features['contains({})'.format(word)] = (word in document_words)
  # count variables for the 4 classes of subjectivity
  weakPos = 0
  strongPos = 0
  weakNeg = 0
  strongNeg = 0
  for word in document_words:
    if word in SL:
      strength, posTag, isStemmed, polarity = SL[word]
      if strength == 'weaksubj' and polarity == 'positive':
        weakPos += 1
      if strength == 'strongsubj' and polarity == 'positive':
        strongPos += 1
      if strength == 'weaksubj' and polarity == 'negative':
        weakNeg += 1
      if strength == 'strongsubj' and polarity == 'negative':
        strongNeg += 1
      features['positivecount'] = weakPos + (2 * strongPos)
      features['negativecount'] = weakNeg + (2 * strongNeg)
  
  if 'positivecount' not in features:
    features['positivecount']=0
  if 'negativecount' not in features:
    features['negativecount']=0      
  return features

In [38]:
###################################################
####   Negation words features  ####
####################################################
# Negation words "not", "never" and "no"
# Not can appear in contractions of the form "doesn", "'", "t"
## if', 'you', 'don', "'", 't', 'like', 'this', 'film', ',', 'then', 'you', 'have', 'a', 'problem', 'with', 'the', 'genre', 'itself', 
# One strategy with negation words is to negate the word following the negation word
#   other strategies negate all words up to the next punctuation
# Strategy is to go through the document words in order adding the word features,
#   but if the word follows a negation words, change the feature to negated word
# Start the feature set with all 2000 word features and 2000 Not word features set to false
negationwords = ['no', 'not', 'never', 'none', 'nowhere', 'nothing', 'noone', 'rather', 'hardly', 'scarcely', 'rarely', 'seldom', 'neither', 'nor']
def NOT_features(document, word_features, negationwords):
  features = {}
  for word in word_features:
    features['contains({})'.format(word)] = False
    features['contains(NOT{})'.format(word)] = False
  # go through document words in order
  for i in range(0, len(document)):
    word = document[i]
    if ((i + 1) < len(document)) and (word in negationwords):
      i += 1
      features['contains(NOT{})'.format(document[i])] = (document[i] in word_features)
    else:
      if ((i + 3) < len(document)) and (word.endswith('n') and document[i+1] == "'" and document[i+2] == 't'):
        i += 3
        features['contains(NOT{})'.format(document[i])] = (document[i] in word_features)
      else:
        features['contains({})'.format(word)] = (word in word_features)
  return features

In [39]:
###################################################
####   POS tag features  ####
####################################################
# this function takes a document list of words and returns a feature dictionay
#   it depends on the variable word_features
# it runs the default pos tagger (the Stanford tagger) on the document
#   and counts 4 types of pos tags to use as features
def POS_features(document, word_features):
    document_words = set(document)
    tagged_words = nltk.pos_tag(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    numNoun = 0
    numVerb = 0
    numAdj = 0
    numAdverb = 0
    for (word, tag) in tagged_words:
        if tag.startswith('N'): numNoun += 1
        if tag.startswith('V'): numVerb += 1
        if tag.startswith('J'): numAdj += 1
        if tag.startswith('R'): numAdverb += 1
    features['nouns'] = numNoun
    features['verbs'] = numVerb
    features['adjectives'] = numAdj
    features['adverbs'] = numAdverb
    return features

In [40]:
#########################################################################################
####  Features   accuracy calculation####
#########################################################################################
def accuracy_calculation(featuresets):
  print ("Training and testing a classifier ")  
  training_size = int(0.1*len(featuresets))
  test_set = featuresets[:training_size]
  training_set = featuresets[training_size:]
  classifier = nltk.NaiveBayesClassifier.train(training_set)
  print ("Accuracy of classifier : ")
  print (nltk.classify.accuracy(classifier, test_set))
  print ("---------------------------------------------------")
  print ("Showing most informative features")
  print (classifier.show_most_informative_features(30))
  print ("---------------------------------------------------")
  print ("precision, recall and F-measure scores")
  print_confusionmatrix(classifier,test_set)
  #print (" ")  

In [41]:
#########################################################################################
## print ConfusionMatrix. ##
#########################################################################################
def print_confusionmatrix(classifier_type, test_set):
  reflist = []
  testlist = []
  for (features, label) in test_set:
    reflist.append(label)
    testlist.append(classifier_type.classify(features))
  
  print (" ")
  print ("The confusion matrix")
  cm = ConfusionMatrix(reflist, testlist)
  print (cm)
  eval_measures(reflist, testlist)

In [42]:
def eval_measures(gold, predicted):
    labels = list(set(gold))
    recall_list = []
    precision_list = []
    F1_list = []
    for lab in labels: 
        TP = FP = FN = TN = 0
        for i, val in enumerate(gold):
            if val == lab and predicted[i] == lab: TP += 1
            if val == lab and predicted[i] != lab: FN += 1
            if val != lab and predicted[i] == lab: FP += 1
            if val != lab and predicted[i] != lab: TN += 1
        recall = TP/(TP + FP)
        precision = TP/(TP + FN)
        recall_list.append(recall)
        precision_list.append(precision)
        F1_list.append(2*(recall*precision)/(recall+precision))
    print("------------------------------------------")
    print('\tPrecision\tRecall\t\tF1')
    print("------------------------------------------")
    for i, lab in enumerate(labels):
        print(lab,'\t' "|{:10.3f}".format(precision_list[i]), "{:10.3f}".format(recall_list[i]), "{:10.3f}".format(F1_list[i]))

In [43]:
def cross_validation_accuracy(num_folds, featuresets):
    subset_size = int(len(featuresets)/num_folds)
    print('Each fold size: {}'.format(subset_size))
    accuracy_list = []
    for i in range(num_folds): 
        test_this_round = featuresets[(i*subset_size):][:subset_size]
        train_this_round = featuresets[:(i*subset_size)]+featuresets[((i+1)*subset_size):]
        classifier = nltk.NaiveBayesClassifier.train(train_this_round)
        accuracy_this_round = nltk.classify.accuracy(classifier, test_this_round)
        print(i, accuracy_this_round)
        accuracy_list.append(accuracy_this_round)
    print('mean accuracy', sum(accuracy_list)/num_folds)

In [44]:
#########################################################################################
####  create test submission file, predict label using classifier ####
#########################################################################################
def create_test_submission(featuresets,test_featuresets,fileName):
  print ("---------------------------------------------------")
  print ("Training and testing a classifier ")
  test_set = test_featuresets
  training_set = featuresets
  classifier = nltk.NaiveBayesClassifier.train(training_set)

  fw = open(fileName,"w")
  fw.write("PhraseId"+','+"Sentiment"+'\n')
  for test,id in test_featuresets:
    fw.write(str(id)+','+str(classifier.classify(test))+'\n')
  fw.close()

In [75]:
#removing punctuation marks from the email messages
def remove_msg_punctuations(email_msg):
    puntuation_removed_msg = "".join([word for word in email_msg if word not in string.punctuation])
    return puntuation_removed_msg

In [76]:
#converting text into lowercase and word tokenizing
def tokenize_into_words(text):
    tokens = re.split('\W+', text)
    return tokens

In [77]:
#lemmatizing
word_lemmatizer = WordNetLemmatizer()
def lemmatization(tokenized_words):
    lemmatized_text = [word_lemmatizer.lemmatize(word)for word in tokenized_words]
    return ' '.join(lemmatized_text)

In [78]:
# create list of mixed spam and ham email documents as (list of words, label)
emaildocs = []
emaildocs_without = []
# add all the ham

#without preprocessing
for ham in ham_finalMailList:
    hamTokensWithOut = nltk.word_tokenize(ham)
    emaildocs_without.append((hamTokensWithOut, 'ham'))

# with pre processing
for ham in ham_finalMailList:
    ham = pre_processing_documents(ham)
    print("email: " + ham + "\n")
    ham = re.sub('\s{2,}', ' ', ham)
    print("email_multiSpaceRemoved: " + ham + "\n")
    ham = lemmatization(tokenize_into_words(remove_msg_punctuations(ham).lower()))
    hsmTokens = nltk.word_tokenize(ham)
    emaildocs.append((hsmTokens, 'ham'))


email: christmas tree farm pictures 

email_multiSpaceRemoved: christmas tree farm pictures 

email: vastar resources  inc  gary  production high island larger block     commenced saturday    p      gross  carlos expects       gross tomorrow  vastar owns   gross production  george x                          forwarded george weissman  hou  ect         

email_multiSpaceRemoved: vastar resources inc gary production high island larger block commenced saturday p gross carlos expects gross tomorrow vastar owns gross production george x forwarded george weissman hou ect 

email:  vastar resources  inc  carlos  please call linda get everything set  ' going estimate    coming tomorrow     increase following day based conversations bill fischer bmar                         forwarded daren j farmer  hou  ect         

email_multiSpaceRemoved:  vastar resources inc carlos please call linda get everything set ' going estimate coming tomorrow increase following day based conversations bill fischer 


email_multiSpaceRemoved:  missing service rates ena hplc cash desk listed expense contracts need rates input sitara please let know soon possible responsible input rates thanks forwarded rhonda robinson hou ect 

email:   missing service rates ena  hplc cash desk okay  expense rates  need help getting input  let know  otherwise ' make sure things clean  th workday  hope great new year '  heidi rhonda robinson          heidi withers  hou  ect  ect  

email_multiSpaceRemoved:  missing service rates ena hplc cash desk okay expense rates need help getting input let know otherwise ' make sure things clean th workday hope great new year ' heidi rhonda robinson heidi withers hou ect ect 

email:   missing service rates ena  hplc cash desk theresa denver looking missing rates issue powder river gathering  thanks help  reminder  beginning close process  th workday  enron north america corp   heidi withers          rhonda robinson  hou  ect  ect  gregory steagall  hou  ect  ect  charlotte hawki

In [79]:
## add all the spam

#without preprocessing
for spam in spam_finalMailList:
    spamTokensWithOut = nltk.word_tokenize(spam)
    emaildocs_without.append((spamTokensWithOut, 'spam'))

# with pre processing
for spam in spam_finalMailList:
    spam = pre_processing_documents(spam)
    print("email: " + spam + "\n")
    spam = re.sub('\s{2,}', ' ', spam)
    print("email_multiSpaceRemoved: " + spam + "\n")
    spam = lemmatization(tokenize_into_words(remove_msg_punctuations(spam).lower()))
    spamTokens = nltk.word_tokenize(spam)
    emaildocs.append((spamTokens, 'spam'))

email: dobmeos hgh energy level gone  stukm introducing doctor  formulated hgh human growth hormone  also called hgh referred medical science master hormone  plentiful young  near age twenty  one bodies begin produce less  time forty nearly everyone deficient hgh  eighty production normally diminished least      advantages hgh   increased muscle strength  loss body fat  increased bone density  lower blood pressure  quickens wound healing  reduces cellulite  improved vision  wrinkle disappearance  increased skin thickness texture  increased energy levels  improved sleep emotional stability  improved memory mental alertness  increased sexual potency  resistance common illness  strengthened heart muscle  controlled cholesterol  controlled mood swings  new hair growth color restore read website unsubscribe 

email_multiSpaceRemoved: dobmeos hgh energy level gone stukm introducing doctor formulated hgh human growth hormone also called hgh referred medical science master hormone plentiful yo

email:  otcbb  itst     uregent news alert urgent news alert stock profile press release international telephone services   telecommunications company born  whose objective bring advanced technology communications within reach clients  started offering call  back service wide portfolio foreigners costa del sol  south spain   extending businesses also based area  december   full liberalisation occurred telecommunications market spain  focused efforts providing made   measure service client best price high level quality  company point offered primarily long distance national international calls  prepaid calling cards  intelligent numbers   x    type   boom telecoms markets mid   began quote otcbb began expanding presence prime tourist locations spain  unfortunately  initiative not enough time get traction dot  com bubble burst dragged investor confidence rest telecommunications sector  end   downsized operations refocused efforts south spain order  ride storm   january   actual sharehold

In [80]:
# randomize the list
random.shuffle(emaildocs)
random.shuffle(emaildocs_without)

In [81]:
for email in emaildocs[:4]:
    print (email)

(['reached', 'around', 'fingered', 'dicking', 'bumbum', 'html', 'head', 'meta', 'http', 'equiv', 'content', 'type', 'content', 'text', 'html', 'charset', 'window', 'meta', 'name', 'generator', 'content', 'microsoft', 'frontpage', 'meta', 'name', 'progid', 'content', 'frontpage', 'editor', 'document', 'titleput', 'right', 'title', 'style', 'type', 'text', 'cs', 'body', 'background', 'image', 'url', 'http', 'nd', 'c', 'backgrou', 'jpg', 'background', 'repeat', 'repeat', 'x', 'style', 'head', 'body', 'style', 'margin', 'div', 'align', 'center', 'center', 'table', 'border', 'cellpadding', 'cellspacing', 'tr', 'td', 'rowspan', 'img', 'border', 'src', 'http', 'nd', 'c', 'captain', 'jpg', 'width', 'height', 'td', 'tdimg', 'border', 'src', 'http', 'nd', 'c', 'captain', 'gif', 'width', 'height', 'td', 'td', 'rowspan', 'img', 'border', 'src', 'http', 'nd', 'c', 'captaino', 'jpg', 'width', 'height', 'td', 'td', 'rowspan', 'img', 'border', 'src', 'http', 'nd', 'c', 'captainl', 'jpg', 'width', 'hei

In [82]:
# possibly filter tokens
normaltokens = get_words_from_phasedocs_normal(emaildocs_without)
preprocessedTokens = get_words_from_phasedocs(emaildocs)

In [83]:
# continue as usual to get all words and create word features
word_features = get_word_features(normaltokens)
featuresets_without_preprocessing = [(normal_features(d, word_features), s) for (d, s) in emaildocs_without]

In [84]:
# print featuresets_without_preprocessing[0]
writeFeatureSets(featuresets_without_preprocessing,"features_normal.csv")
print ("---------------------------------------------------")
print ("Accuracy with normal features, without pre-processing steps : ")
accuracy_calculation(featuresets_without_preprocessing)
print ("\n\n-----------------------CV--------------------------")
num_folds = 5
cross_validation_accuracy(num_folds, featuresets_without_preprocessing)

---------------------------------------------------
Accuracy with normal features, without pre-processing steps : 
Training and testing a classifier 
Accuracy of classifier : 
0.8888888888888888
---------------------------------------------------
Showing most informative features
Most Informative Features
            contains(cc) = True              ham : spam   =     17.3 : 1.0
            contains(99) = True              ham : spam   =     15.0 : 1.0
             contains(*) = True             spam : ham    =     14.1 : 1.0
             contains(%) = True             spam : ham    =     13.5 : 1.0
        contains(thanks) = True              ham : spam   =     12.7 : 1.0
          contains(deal) = True              ham : spam   =     11.9 : 1.0
         contains(hours) = True             spam : ham    =     10.9 : 1.0
          contains(corp) = True              ham : spam   =      9.9 : 1.0
            contains(12) = True              ham : spam   =      9.9 : 1.0
             conta

In [85]:
word_features = get_word_features(preprocessedTokens)
#print word_features[:20]
featuresets = [(normal_features(d, word_features), s) for (d, s) in emaildocs]
#print featuresets[0]
writeFeatureSets(featuresets,"features_preprocessed.csv")
print ("---------------------------------------------------")
print ("Accuracy with pre-processed features : ")
accuracy_calculation(featuresets)
print ("\n\n-----------------------CV--------------------------")
num_folds = 5
cross_validation_accuracy(num_folds, featuresets)

---------------------------------------------------
Accuracy with pre-processed features : 
Training and testing a classifier 
Accuracy of classifier : 
0.9629629629629629
---------------------------------------------------
Showing most informative features
Most Informative Features
        contains(online) = True             spam : ham    =     14.5 : 1.0
        contains(thanks) = True              ham : spam   =     12.7 : 1.0
         contains(today) = True             spam : ham    =     11.5 : 1.0
          contains(name) = True             spam : ham    =     11.0 : 1.0
          contains(hour) = True             spam : ham    =     11.0 : 1.0
         contains(offer) = True             spam : ham    =     10.8 : 1.0
       contains(message) = True             spam : ham    =     10.1 : 1.0
      contains(security) = True             spam : ham    =      9.9 : 1.0
          contains(deal) = True              ham : spam   =      9.3 : 1.0
       contains(content) = True          

In [86]:
SL_featuresets = [(SL_features(d, word_features, SL), c) for (d, c) in emaildocs]
writeFeatureSets(SL_featuresets,"features_SL.csv")
#print SL_featuresets[0]
print ("---------------------------------------------------")
print ("Accuracy with SL_featuresets : ")
accuracy_calculation(SL_featuresets)
print ("\n\n-----------------------CV--------------------------")
num_folds = 5
cross_validation_accuracy(num_folds, SL_featuresets)

---------------------------------------------------
Accuracy with SL_featuresets : 
Training and testing a classifier 
Accuracy of classifier : 
0.9259259259259259
---------------------------------------------------
Showing most informative features
Most Informative Features
        contains(online) = True             spam : ham    =     14.5 : 1.0
        contains(thanks) = True              ham : spam   =     12.7 : 1.0
         contains(today) = True             spam : ham    =     11.5 : 1.0
          contains(name) = True             spam : ham    =     11.0 : 1.0
          contains(hour) = True             spam : ham    =     11.0 : 1.0
         contains(offer) = True             spam : ham    =     10.8 : 1.0
       contains(message) = True             spam : ham    =     10.1 : 1.0
      contains(security) = True             spam : ham    =      9.9 : 1.0
          contains(deal) = True              ham : spam   =      9.3 : 1.0
       contains(content) = True             spam 

In [87]:
NOT_featuresets = [(NOT_features(d, word_features, negationwords), c) for (d, c) in emaildocs]
#print NOT_featuresets[0]
writeFeatureSets(SL_featuresets,"features_NOT.csv")
print ("---------------------------------------------------")
print ("Accuracy with NOT_featuresets : ")
accuracy_calculation(NOT_featuresets)
print ("\n\n-----------------------CV--------------------------")
num_folds = 5
cross_validation_accuracy(num_folds, NOT_featuresets)

---------------------------------------------------
Accuracy with NOT_featuresets : 
Training and testing a classifier 
Accuracy of classifier : 
1.0
---------------------------------------------------
Showing most informative features
Most Informative Features
             contains(r) = False            spam : ham    =     16.9 : 1.0
        contains(online) = True             spam : ham    =     14.5 : 1.0
            contains(pm) = False             ham : spam   =     14.3 : 1.0
        contains(thanks) = True              ham : spam   =     12.7 : 1.0
             contains(g) = False            spam : ham    =     12.2 : 1.0
         contains(today) = True             spam : ham    =     11.5 : 1.0
          contains(hour) = True             spam : ham    =     11.0 : 1.0
          contains(name) = True             spam : ham    =     11.0 : 1.0
         contains(offer) = True             spam : ham    =     10.8 : 1.0
       contains(message) = True             spam : ham    =    

In [88]:
bigram_features = get_biagram_features(preprocessedTokens)
print (bigram_features[0])
bigram_featuresets = [(bigram_document_features(d, word_features,bigram_features), c) for (d, c) in emaildocs]
#print bigram_featuresets[0]
writeFeatureSets(bigram_featuresets,"features_biagram.csv")
print ("---------------------------------------------------")
print ("Accuracy with bigram featuresets : ")
accuracy_calculation(bigram_featuresets)
print ("\n\n-----------------------CV--------------------------")
num_folds = 5
cross_validation_accuracy(num_folds, bigram_featuresets)

--------------frequency score--------------
(('hou', 'ect'), 0.007945106536655833)
(('ect', 'ect'), 0.003637207862559975)
(('corp', 'enron'), 0.0011350152195222618)
(('width', 'height'), 0.0010834236186348862)
(('let', 'know'), 0.0009544446164164474)
(('hour', 'hour'), 0.0008254656141980086)
(('full', 'story'), 0.0007480782128669452)
(('ami', 'chokshi'), 0.0006964866119795697)
(('http', 'www'), 0.0006964866119795697)
(('january', 'full'), 0.0006964866119795697)
(('january', 'story'), 0.0006964866119795697)
(('align', 'center'), 0.0006448950110921942)
(('enron', 'enron'), 0.0006448950110921942)
(('src', 'http'), 0.0006448950110921942)
(('border', 'http'), 0.0006190992106485065)
(('border', 'src'), 0.0006190992106485065)
(('chokshi', 'enron'), 0.0006190992106485065)
(('ami', 'corp'), 0.0005933034102048187)
(('chokshi', 'corp'), 0.0005933034102048187)
(('please', 'let'), 0.0004901202084300676)
(('ect', 'mary'), 0.0004643244079863798)
(('international', 'call'), 0.0004643244079863798)
(('m

In [89]:
pos_featuresets = [(POS_features(d, word_features), c) for (d, c) in emaildocs]
print(pos_featuresets[0])
writeFeatureSets(pos_featuresets,"features_pos.csv")
print("---------------------------------------------------")
print("Accuracy with pos_featuresets : ")
writeFeatureSets(pos_featuresets,"features_pos_featuresets.csv")
accuracy_calculation(pos_featuresets)
print ("\n\n-----------------------CV--------------------------")
num_folds = 5
cross_validation_accuracy(num_folds, pos_featuresets)

({'contains(ect)': False, 'contains(hou)': False, 'contains(not)': False, 'contains(please)': True, 'contains(gas)': False, 'contains(meter)': False, 'contains(http)': True, 'contains(deal)': False, 'contains(enron)': False, 'contains(call)': True, 'contains(january)': False, 'contains(can)': False, 'contains(need)': False, 'contains(thanks)': False, 'contains(new)': True, 'contains(contract)': False, 'contains(know)': False, 'contains(width)': True, 'contains(volume)': False, 'contains(service)': True, 'contains(month)': False, 'contains(forwarded)': False, 'contains(day)': False, 'contains(let)': True, 'contains(work)': False, 'contains(rate)': False, 'contains(get)': True, 'contains(hpl)': False, 'contains(height)': True, 'contains(com)': False, 'contains(information)': False, 'contains(back)': True, 'contains(email)': False, 'contains(corp)': False, 'contains(full)': False, 'contains(company)': False, 'contains(production)': False, 'contains(ami)': False, 'contains(question)': Fals

1 0.8703703703703703
2 0.8888888888888888
3 0.9444444444444444
4 0.9074074074074074
mean accuracy 0.9148148148148149


In [72]:
# possibly filter tokens/stopwords/lemmitization (Debasis) (Done)

# different sizes of vocabularies/Corpus statistics -ngram/bigram (Week2) (Debasis) (Done)

# continue as usual to get all words and create word features (Debasis) (Done)

# train classifier and show performance in cross-validation or NaiveBayes (classifier score, precision, recall, F1-score etc.(Debasis) (Done)
  
# representing negation (Week8) (Debasis) (Done)

# sentiment lexicon with scores or counts: Subjectivity (Week7)(Debasis) (Done)

# POS tagging (Week9) (Debasis) (Done)

# Documentation (Draft Ready)